<a href="https://colab.research.google.com/github/Nandini37/Python_ML_AI/blob/main/FraudDetectionTransformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import polars as pl
warnings.filterwarnings('ignore')

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=5dcc5e89c91bb58cdb68eec076f609a15096c404160f030ab82ac12ddb9a34f5
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [47]:
df = pl.read_csv('/content/Fraud.csv')
print(df.shape)

(6362620, 11)


In [ ]:
df.head()

step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
i64,str,f64,str,f64,f64,str,f64,f64,i64,i64
1,"""PAYMENT""",9839.64,"""C1231006815""",170136.0,160296.36,"""M1979787155""",0.0,0.0,0,0
1,"""PAYMENT""",1864.28,"""C1666544295""",21249.0,19384.72,"""M2044282225""",0.0,0.0,0,0
1,"""TRANSFER""",181.0,"""C1305486145""",181.0,0.0,"""C553264065""",0.0,0.0,1,0
1,"""CASH_OUT""",181.0,"""C840083671""",181.0,0.0,"""C38997010""",21182.0,0.0,1,0
1,"""PAYMENT""",11668.14,"""C2048537720""",41554.0,29885.86,"""M1230701703""",0.0,0.0,0,0


In [ ]:

for col in df.columns:
  if df[col].dtype == pl.Utf8:

    print(f"Column name: {col}")
    print(f"Unique values for column '{col}':")
    print(df.select(pl.col(col).unique()).to_pandas())
    print("\n")

Column name: type
Unique values for column 'type':
       type
0   PAYMENT
1     DEBIT
2  TRANSFER
3   CASH_IN
4  CASH_OUT


Column name: nameOrig
Unique values for column 'nameOrig':
            nameOrig
0          C39084032
1         C202636861
2        C1952421932
3        C2096952830
4        C1004188084
...              ...
1643708  C1376510884
1643709  C1855827409
1643710   C373229243
1643711  C2062004185
1643712  C1253751378

[1643713 rows x 1 columns]


Column name: nameDest
Unique values for column 'nameDest':
           nameDest
0        C935053787
1        C946363477
2        M612574500
3        C388395512
4        M269670299
...             ...
700856  M1480634019
700857     M8945663
700858   M150695234
700859  M1710459937
700860   M104138594

[700861 rows x 1 columns]




In [19]:
df.describe()

statistic,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
str,f64,str,f64,str,f64,f64,str,f64,f64,f64,f64
"""count""",6.36262e6,"""6362620""",6.36262e6,"""6362620""",6.36262e6,6.36262e6,"""6362620""",6.36262e6,6.36262e6,6.36262e6,6.36262e6
"""null_count""",0.0,"""0""",0.0,"""0""",0.0,0.0,"""0""",0.0,0.0,0.0,0.0
"""mean""",243.397246,null,179861.903549,null,833883.104074,855113.668579,null,1.1007e6,1.2250e6,0.001291,0.000003
"""std""",142.331971,null,603858.231463,null,2.8882e6,2.9240e6,null,3.3992e6,3.6741e6,0.035905,0.001586
"""min""",1.0,"""CASH_IN""",0.0,"""C1000000639""",0.0,0.0,"""C1000004082""",0.0,0.0,0.0,0.0
"""25%""",156.0,null,13389.57,null,0.0,0.0,null,0.0,0.0,0.0,0.0
"""50%""",239.0,null,74872.08,null,14208.0,0.0,null,132705.81,214661.65,0.0,0.0
"""75%""",335.0,null,208721.45,null,107315.0,144258.41,null,943036.53,1.1119e6,0.0,0.0
"""max""",743.0,"""TRANSFER""",9.2446e7,"""C999999784""",5.9585e7,4.9585e7,"""M999999784""",3.5602e8,3.5618e8,1.0,1.0


In [21]:
# Get columns with float and int data types
float_int_cols = [col for col, dtype in df.schema.items() if dtype in (pl.Float32, pl.Float64, pl.Int8, pl.Int16, pl.Int32, pl.Int64)]

# Calculate min and max for each selected column
for col in float_int_cols:
    min_val = df[col].min()
    max_val = df[col].max()
    mean_value = df[col].mean()
    percentile75 = df[col].quantile(0.75)
    percentile25 = df[col].quantile(0.25)
    print(f"Column: {col}")
    print(f"Minimum value: {min_val}")
    print(f"Maximum value: {max_val}")
    print(f"Mean value: {mean_value}")
    print(f"75th percentile: {percentile75}")
    print(f"25th percentile: {percentile25}")
    print("\n")

Column: step
Minimum value: 1
Maximum value: 743
Mean value: 243.39724563151657
75th percentile: 335.0
25th percentile: 156.0


Column: amount
Minimum value: 0.0
Maximum value: 92445516.64
Mean value: 179861.90354913071
75th percentile: 208721.45
25th percentile: 13389.57


Column: oldbalanceOrg
Minimum value: 0.0
Maximum value: 59585040.37
Mean value: 833883.1040744866
75th percentile: 107315.0
25th percentile: 0.0


Column: newbalanceOrig
Minimum value: 0.0
Maximum value: 49585040.37
Mean value: 855113.6685785886
75th percentile: 144258.41
25th percentile: 0.0


Column: oldbalanceDest
Minimum value: 0.0
Maximum value: 356015889.35
Mean value: 1100701.666519649
75th percentile: 943036.53
25th percentile: 0.0


Column: newbalanceDest
Minimum value: 0.0
Maximum value: 356179278.92
Mean value: 1224996.3982019294
75th percentile: 1111909.16
25th percentile: 0.0


Column: isFraud
Minimum value: 0
Maximum value: 1
Mean value: 0.001290820448180152
75th percentile: 0.0
25th percentile: 0.0




In [30]:


q = (
    df.lazy()
    .group_by("isFraud")  # Group by 'isFraud'
    .agg(pl.count().alias("count"))  # Count occurrences
    .sort("count", descending=True)  # Sort by count in descending order
)

result_df = q.collect()  # Execute the query and get the result DataFrame
print(result_df)

shape: (2, 2)
┌─────────┬─────────┐
│ isFraud ┆ count   │
│ ---     ┆ ---     │
│ i64     ┆ u32     │
╞═════════╪═════════╡
│ 0       ┆ 6354407 │
│ 1       ┆ 8213    │
└─────────┴─────────┘


In [33]:


# Set the display options to show all columns and rows
#pl.Config.set_tbl_rows(None)  # Set to None to display all rows
pl.Config.set_tbl_cols(None)  # Set to None to display all columns

print(df)

shape: (6_362_620, 11)
┌──────┬──────────┬───────────┬─────────────┬───┬─────────────┬─────────────┬─────────┬────────────┐
│ step ┆ type     ┆ amount    ┆ nameOrig    ┆ … ┆ oldbalanceD ┆ newbalanceD ┆ isFraud ┆ isFlaggedF │
│ ---  ┆ ---      ┆ ---       ┆ ---         ┆   ┆ est         ┆ est         ┆ ---     ┆ raud       │
│ i64  ┆ str      ┆ f64       ┆ str         ┆   ┆ ---         ┆ ---         ┆ i64     ┆ ---        │
│      ┆          ┆           ┆             ┆   ┆ f64         ┆ f64         ┆         ┆ i64        │
╞══════╪══════════╪═══════════╪═════════════╪═══╪═════════════╪═════════════╪═════════╪════════════╡
│ 1    ┆ PAYMENT  ┆ 9839.64   ┆ C1231006815 ┆ … ┆ 0.0         ┆ 0.0         ┆ 0       ┆ 0          │
│ 1    ┆ PAYMENT  ┆ 1864.28   ┆ C1666544295 ┆ … ┆ 0.0         ┆ 0.0         ┆ 0       ┆ 0          │
│ 1    ┆ TRANSFER ┆ 181.0     ┆ C1305486145 ┆ … ┆ 0.0         ┆ 0.0         ┆ 1       ┆ 0          │
│ 1    ┆ CASH_OUT ┆ 181.0     ┆ C840083671  ┆ … ┆ 21182.0     ┆ 0.0 

In [48]:
# prompt: Handle missing values in pl df. I want to see all column output. the output is truc

# Assuming df is your Polars DataFrame
for col in df.columns:
  # Check for missing values in each column
  if df[col].null_count() > 0:
    print(f"Column '{col}' has missing values.")
  else:
    print(f"Column '{col}' has no missing values.")





Column 'step' has no missing values.
Column 'type' has no missing values.
Column 'amount' has no missing values.
Column 'nameOrig' has no missing values.
Column 'oldbalanceOrg' has no missing values.
Column 'newbalanceOrig' has no missing values.
Column 'nameDest' has no missing values.
Column 'oldbalanceDest' has no missing values.
Column 'newbalanceDest' has no missing values.
Column 'isFraud' has no missing values.
Column 'isFlaggedFraud' has no missing values.


In [49]:
# prompt: handle outliners in data return the count of values replaced, and % of totalcol values replaced

def handle_outliers(df, col, threshold=1.5):
  """
  Handles outliers in a specified column of a Polars DataFrame using the IQR method.

  Args:
    df: The Polars DataFrame.
    col: The name of the column to handle outliers in.
    threshold: The multiplier for the IQR to define outlier boundaries (default is 1.5).

  Returns:
    A tuple containing:
      - The modified DataFrame with outliers replaced by the nearest boundary value.
      - The count of values replaced.
      - The percentage of total column values replaced.
  """

  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - threshold * iqr
  upper_bound = q3 + threshold * iqr

  # Replace outliers with the nearest boundary value
  df_modified = df.with_columns(
      pl.when(pl.col(col) < lower_bound)
      .then(lower_bound)
      .when(pl.col(col) > upper_bound)
      .then(upper_bound)
      .otherwise(pl.col(col))
      .alias(col)
  )

  # Count values replaced
  count_replaced = df.filter((pl.col(col) < lower_bound) | (pl.col(col) > upper_bound)).shape[0]

  # Calculate percentage of total column values replaced
  percentage_replaced = (count_replaced / df.shape[0]) * 100 if df.shape[0] > 0 else 0

  return df_modified, count_replaced, percentage_replaced


# Example usage for a specific column (replace 'your_column_name' with your column)
# df_modified, count_replaced, percentage_replaced = handle_outliers(df, 'your_column_name')

# Print the results
# print(f"Count of values replaced in 'your_column_name': {count_replaced}")
# print(f"Percentage of total values replaced in 'your_column_name': {percentage_replaced:.2f}%")


In [50]:
print(df.columns)

['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']


In [51]:
df_modified, count_replaced, percentage_replaced = handle_outliers(df, 'oldbalanceOrg')
# print(f"Count of values replaced in 'your_column_name': {count_replaced}")
# print(f"Percentage of total values replaced in 'your_column_name': {percentage_replaced:.2f}%")


In [52]:
print(f"Count of values replaced in 'your_column_name': {count_replaced}")
print(f"Percentage of total values replaced in 'your_column_name': {percentage_replaced:.2f}%")


Count of values replaced in 'your_column_name': 1112508
Percentage of total values replaced in 'your_column_name': 17.49%
